In [86]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
from urllib.parse import urljoin
import pandas as pd
from tqdm.notebook import tqdm
import time
import math

In [77]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=187940'
html = urlopen(url_base + url_sub)
soup = BeautifulSoup(html,'html.parser')

In [78]:
#iframe의 src주소 값을 얻어야함
ifr_url = soup.find('iframe')['src']
ifr_url

'/movie/bi/mi/pointWriteFormList.nhn?code=187940&type=after&isActualPointWriteExecute=false&isMileageSubscriptionAlready=false&isMileageSubscriptionReject=false'

In [206]:
url = url_base + ifr_url + '&page='
page = 1
html = urlopen(url + str(page))
soup = BeautifulSoup(html,'html.parser')

In [85]:
total_review = int(soup.find('strong','total').find('em').get_text().replace(',',''))
total_review

30257

In [75]:
total_review = soup.find('strong','total')
total_review

<strong class="total">관람객 평점 <em>4,813</em>건<button class="btn_review" id="open-form-btn">내 평점 등록</button></strong>

In [245]:
loop_count = math.ceil(total_review/10)
loop_count

3026

In [102]:
review_lists = soup.find(class_ ='score_result').find_all('li')
review_lists

[<li>
 <div class="star_score">
 <span class="st_off"><span class="st_on" style="width:10.0%"></span></span><em>1</em>
 </div>
 <div class="score_reple">
 <p>
 <!-- 스포일러 컨텐츠로 처리되는지 여부 -->
 <span id="_filtered_ment_0">
 														
 															
 															
 																우리나라 영화는 왜 미군이 악 북한이 선인거냐? 
 															
 														
 														
 													</span>
 </p>
 <dl>
 <dt>
 <em>
 <a href="#" onclick="javascript:showPointListByNid(16497563, 'after');parent.clickcr(this, 'ara.uid', '', '', event); return false;" target="_top">
 <span>PAC3MSE(aegi****)</span>
 </a>
 </em>
 <em>2019.12.19 11:45</em>
 </dt>
 <dd>
 <a class="go_report2" href="#" onclick="parent.clickcr(this, 'ara.report', '', '', event); common.report('false','aegi****', 'uw0xwCWs/0QeKdI9DvRGbTQ5ZFJbXD0kzCe71x1zs1M=', '우리나라 영화는 왜 미군이 악 북한이 선인거냐? ', '16497563', 'point_after', false);return false;"><em>신고</em></a>
 </dd>
 </dl>
 </div>
 <div class="btn_area">
 <a class="_sympathyButton" href="#" oncli

In [111]:
score = review_lists[0].find(class_='star_score').find('em').get_text()
score

'1'

In [143]:
date = review_lists[0].find_all('em')[2].get_text()
date

'2019.12.19 11:45'

In [151]:
identify = review_lists[0].find('dt').find('span').get_text()
identify

'PAC3MSE(aegi****)'

In [160]:
reple_basic = review_lists[0].find('p').get_text().strip()
reple_basic

'우리나라 영화는 왜 미군이 악 북한이 선인거냐?'

In [ ]:
reple_viewer = review_lists[3].find('p').get_text().replace('관람객','').strip('\n\t\r')
reple_viewer

In [191]:
reple_fold = review_lists[4].find(class_ = '_unfold_ment').find('a')['data-src']
reple_fold

'역대급 출연진에 기대 많이 하고 봤는데 참 실망스럽네요. 미군은 완전 악의축으로 묘사하고 중책을 맡은 남한 특수부대원들은 오합지졸이며 멋있는 인물은 오직 북한군 한명 뿐 ㅋㅋㅋ 완전 정치적 목적이 다분한 그런 설정들은 둘째치더라도 개연성도 엄청 떨어지고 재미가 없어요. 저 출연진과 그정도 제작비로 이딴 영화밖에 못만드는게 국내영화의 한계인가요? 창피한줄 아세요 '

In [ ]:
reple_spoil = review_lists[8].find(class_ = '_unfold_ment').find('a')['data-src']
reple_spoil

In [246]:
url_base = 'https://movie.naver.com'
url_sub = '/movie/bi/mi/point.nhn?code=187940'
html = urlopen(url_base + url_sub)
soup = BeautifulSoup(html,'html.parser')

ifr_url = soup.find('iframe')['src']
ifr_url

score = []
date = []
identify = []
reple = []
for i in tqdm(range(1,loop_count)):
    url = url_base + ifr_url + '&page='
    page = i
    html = urlopen(url + str(page))
    soup = BeautifulSoup(html,'html.parser')
    review_lists = soup.find(class_ ='score_result').find_all('li')
    
    for review_list in review_lists:
    
        score.append(review_list.find(class_='star_score').find('em').get_text())
        date.append(review_list.find_all('em')[2].get_text())
        identify.append(review_list.find('dt').find('span').get_text())

        fold = review_list.find(class_ = '_unfold_ment')
        viewer = review_list.find(class_ = 'ico_viewer')
        spoil = review_list.find(class_ = 'text_spo _text_spo')
        
        if fold != None:
            reple.append(review_list.find(class_ = '_unfold_ment').find('a')['data-src'])
        elif viewer != None or spoil != None:
            reple.append(review_list.find('p').get_text().replace('관람객','').replace('스포일러가 포함된 감상평입니다. 감상평 보기','').strip('\n\t\r'))
        else :
            reple.append(review_list.find('p').get_text().strip())


In [247]:
df = pd.DataFrame(dict(zip(['평점','날짜','아이디','리플'],[score,date,identify,reple])))
df

,평점,날짜,아이디,리플
0,1,2019.12.19 11:45,PAC3MSE(aegi****),우리나라 영화는 왜 미군이 악 북한이 선인거냐?
1,2,2019.12.19 13:16,The Moon(dark****),저 배우들을 가져다가 이렇게 밖에 못만드냐? 대체 뭘 보여주고 싶은거야? 그냥 기존...
2,1,2019.12.19 14:18,Synt(ace2****),제발 이제는 북한관련 영화좀 안나왔으면....특히 왜 우리 우방인 미국은 항상 적으...
3,1,2019.12.19 14:51,sida****,북한 착한척 하는 영화 좀 그만 만들어요
4,1,2019.12.20 01:50,BrownEM(tupd****),역대급 출연진에 기대 많이 하고 봤는데 참 실망스럽네요. 미군은 완전 악의축으로 묘...
...,...,...,...,...
30245,1,2020.01.21 23:17,hello(hjkj****),솔직히 진짜 재미없어요 보지마세요 영화가 수준이 낮아요 그리고 좌빨들 제발 대한민국...
30246,1,2020.01.13 08:00,빈배(binb****),개돼지들아 북한을 찬양하라.
30247,1,2020.01.14 20:14,슬픈향기(eldo****),친북좌익 좌파영화 혐오한다.
30248,1,2020.01.04 13:19,Paul(rer0****),북한이 우리의 주적이자 악이다. 미국이 아니라


In [248]:
df.to_csv('Movie_score_2.csv',sep=',',encoding='utf-8')

['PAC3MSE(aegi****)',
 'The Moon(dark****)',
 'Synt(ace2****)',
 'sida****',
 'BrownEM(tupd****)',
 '에코(ruri****)',
 '1029309(tjdd****)',
 '말과글(7111****)',
 '안경잡이(dksg****)',
 '김안개(hojo****)',
 '에반탱크(sabe****)',
 'kiss****',
 '권도형(kdof****)',
 'Baileys(kang****)',
 'djxy****',
 'smile(la21****)',
 '탱글이(tang****)',
 '범부의담(sumg****)',
 '열사입니다(kim8****)',
 'Torysastro(tory****)',
 'HJK(skyl****)',
 'NDDD(nddd****)',
 '스피노자(gusx****)',
 'hygge(nams****)',
 '위드미(skym****)',
 '5900****',
 'dark****',
 '그래스Wo(swee****)',
 '고르뎅(ipla****)',
 '다다오(vndt****)',
 '출이(sush****)',
 '칩사마(ppda****)',
 '냉동상싯두(tldn****)',
 'thdt****',
 'wh(whan****)',
 'shaz****',
 'osw7****',
 '강민준(theb****)',
 'LTill(smar****)',
 '나영흠(qwas****)',
 '끌림(9780****)',
 'simc****',
 '오렌지탱자(yere****)',
 '파이팅맨(rive****)',
 'Fucking bitch(hoho****)',
 'vict****',
 'Wjddntjd(jemi****)',
 'last****',
 '음바페의 음바음바(hjk0****)',
 '08태훈(kthk****)',
 'SaG(dhwk****)',
 'dbuuu(dadu****)',
 'practical gear(anmi****)',
 '오덕후(sein****)',
 'H

['우리나라 영화는 왜 미군이 악 북한이 선인거냐?',
 '저 배우들을 가져다가 이렇게 밖에 못만드냐? 대체 뭘 보여주고 싶은거야? 그냥 기존의 대작들 다 범벅켜서 이맛도 저맛도아닌 영화. 재난영화도아냐 블록버스터도아냐 첩보물도 아냐 신파코드는 뭐야? 드럽게 못만드네 그 자본으로 ',
 '제발 이제는 북한관련 영화좀 안나왔으면....특히 왜 우리 우방인 미국은 항상 적으로 나오냐 영화 선',
 '북한 착한척 하는 영화 좀 그만 만들어요',
 '역대급 출연진에 기대 많이 하고 봤는데 참 실망스럽네요. 미군은 완전 악의축으로 묘사하고 중책을 맡은 남한 특수부대원들은 오합지졸이며 멋있는 인물은 오직 북한군 한명 뿐 ㅋㅋㅋ 완전 정치적 목적이 다분한 그런 설정들은 둘째치더라도 개연성도 엄청 떨어지고 재미가 없어요. 저 출연진과 그정도 제작비로 이딴 영화밖에 못만드는게 국내영화의 한계인가요? 창피한줄 아세요 ',
 '시간 가는 줄 알고 봤습니다.',
 '엄복동 , 염력은 피했는데 백두산을 못피했네 ..',
 '미군이 적이라고?? 종북 out',
 '알바 너무 많은거 아냐?',
 '지질학 전공생입니다 왜 우리나라는 맨날 영화만들때 과학적 고증은 개무시하고 만드나요?',
 '님들아.... 차라리 포드페라리보세요 !!!!',
 '영상미는 좋았으나..스토리나 감동이없어요 조금지루하면도 있었구요 뭘알리고자하는건지 급하게백두산만터지고. 스릴 감동 슬픔 셋중에하나라도 느끼게해주시던지..많은걸 바랬나봅니다...  개인차이가 있을지는모르겠으나 ,보신분들은 공감하실듯....영상미뽑는데만 열중하신듯요..좋은배우분들이 이영화를 선택하신게..참.. ',
 '와... 내돈..',
 '소재와 배우가 아깝다 어떻게 이런... ',
 '개봉전부터 홍보를 많이해서 친구들과 보고왔는데..초반에 초호화 배우진에 몰입해서 보다가..중반부터는..왜 백두산이 소재가되었는데도 내용이 산으로 가는 느낌이드는건지배우들에 케미는 좋지만 배우들 케미보려고 영화보는게아닌스토리를 보기위해 보는건데도뭔가 화려한거만 강조하는

In [208]:
score = []
date = []
identify = []
reple = []

for review_list in review_lists:
    
    score.append(review_list.find(class_='star_score').find('em').get_text())
    date.append(review_list.find_all('em')[2].get_text())
    identify.append(review_list.find('dt').find('span').get_text())
    
    fold = review_list.find(class_ = '_unfold_ment')
    viewer = review_list.find(class_ = '_unfold_ment')
    spoil = review_list.find(class_ = 'text_spo _text_spo')
    if fold != None:
        reple.append(review_list.find(class_ = '_unfold_ment').find('a')['data-src'])
    elif viewer != None:
        reple.append(review_list.find('p').get_text().replace('관람객','').strip('\n\t\r'))
    else :
        reple.append(review_list.find('p').get_text().strip())
